# trait

> Fill in a module description here

In [1]:
#| default_exp trait

In [2]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import List, Optional

import jax
import jax.numpy as jnp
from flax.struct import dataclass as flax_dataclass
from jaxtyping import Array, Float, Int, PyTree

# Assuming Population and SimParam are in these locations
from chewc.population import Population
from chewc.sp import SimParam

# --- Base Loci and Trait Structures ---

@flax_dataclass(frozen=True)
class LociMap:
    loci_per_chr: Int[Array, "nChr"]
    loci_loc: Int[Array, "nLoci"]
    name: str

    @property
    def n_loci(self) -> int:
        return self.loci_loc.shape[0]

@flax_dataclass(frozen=True)
class TraitA(LociMap):
    add_eff: Float[Array, "nLoci"]
    intercept: float = 0.0

@flax_dataclass(frozen=True)
class TraitCollection:
    loci_loc: Int[Array, "nLoci"]
    add_eff: Float[Array, "nTraits nLoci"]
    intercept: Float[Array, "nTraits"]

    @property
    def n_traits(self) -> int:
        return self.add_eff.shape[0]

    @property
    def n_loci(self) -> int:
        return self.loci_loc.shape[0]

# --- Helper Functions ---

def _calculate_gvs_vectorized_alternative(
    pop: Population,
    traits: TraitCollection,
    ploidy: int
) -> Float[Array, "nInd nTraits"]:
    """Calculates all genetic values using a single matrix multiplication."""
    flat_geno_alleles = pop.geno.transpose((0, 1, 3, 2)).reshape(pop.nInd, -1, ploidy)
    qtl_alleles = flat_geno_alleles[:, traits.loci_loc, :]
    qtl_geno = jnp.sum(qtl_alleles, axis=2)
    all_bv = jnp.dot(qtl_geno, traits.add_eff.T)
    all_gvs = all_bv + traits.intercept
    return all_gvs

# --- Public Trait-Adding Function ---
def add_trait_a(
    key: jax.random.PRNGKey,
    sim_param: SimParam,
    n_qtl_per_chr: int,
    mean: Float[Array, "nTraits"],
    var: Float[Array, "nTraits"],
    cor_a: Optional[Float[Array, "nTraits nTraits"]] = None
) -> SimParam:
    """Adds one or more new additive traits to the simulation parameters."""
    n_traits = mean.shape[0]
    assert mean.shape == var.shape, "Mean and variance vectors must have the same shape."
    if cor_a is None:
        cor_a = jnp.identity(n_traits)

    key, sample_key, qtl_key = jax.random.split(key, 3)

    n_total_qtl = n_qtl_per_chr * sim_param.n_chr
    all_loci_indices = jnp.arange(sim_param.gen_map.size)
    qtl_loc = jax.random.choice(qtl_key, all_loci_indices, shape=(n_total_qtl,), replace=False)
    qtl_loc = jnp.sort(qtl_loc)

    raw_effects = jax.random.normal(sample_key, (n_total_qtl, n_traits))
    cholesky_matrix = jnp.linalg.cholesky(cor_a)
    correlated_raw_effects = jnp.dot(raw_effects, cholesky_matrix.T)

    all_scaled_effects = []
    all_intercepts = []

    # Create a temporary TraitCollection to calculate initial variance
    temp_traits = TraitCollection(
        loci_loc=qtl_loc,
        add_eff=correlated_raw_effects.T, # Transpose to (nTraits, nQtl)
        intercept=jnp.zeros(n_traits)
    )
    
    # Calculate initial breeding values and variance on the founder population
    initial_bvs = _calculate_gvs_vectorized_alternative(sim_param.founderPop, temp_traits, sim_param.ploidy)
    initial_vars = jnp.var(initial_bvs, axis=0)

    # Vectorized calculation of scaling factors and intercepts
    scaling_factors = jnp.sqrt(var / (initial_vars + 1e-8))
    initial_means = jnp.mean(initial_bvs, axis=0)
    final_intercepts = mean - (initial_means * scaling_factors)
    
    # Scale all effects at once using broadcasting
    final_add_eff = correlated_raw_effects * scaling_factors # (nQtl, nTraits) * (nTraits,)

    trait_collection = TraitCollection(
        loci_loc=qtl_loc,
        add_eff=final_add_eff.T, # Store as (nTraits, nQtl)
        intercept=final_intercepts
    )

    return sim_param.replace(traits=trait_collection)



In [4]:
#| hide
import nbdev; nbdev.nbdev_export()